In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用模型监控的 Vertex AI 批量预测

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform/%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fbatch_prediction_model_monitoring.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb" target='_blank'>
       <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

在这本笔记本中，您将学习如何在部署的 Vertex AI 模型资源上使用批处理预测请求进行模型监控。在配套的笔记本中，<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb" target="_blank">Vertex AI Model Monitoring with Explainable AI feature attributions</a>，您可以了解如何将模型监控应用到流式实时预测中。

了解有关[Vertex AI 模型监控批处理预测](https://cloud.google.com/vertex-ai/docs/model-monitoring/model-monitoring-batch-predictions)的更多信息。

### 目标
在本笔记本中，您将学习如何使用Vertex AI模型监控服务来检测批量预测中的漂移和异常。

本教程使用以下谷歌云ML服务：

- Vertex AI模型监控
- Vertex AI批量预测
- Vertex AI模型资源

执行的步骤包括：

- 将预训练模型上传为Vertex AI模型资源。
- 生成批量预测请求。
- 解释模型监控功能报告的统计数据、可视化和其他数据。

### 模型

该教程使用一个预训练的模型，模型的工件存储在一个公共云存储桶中。该模型用于预测在线游戏网站上玩家可能流失的概率，即停止成为活跃玩家。

### 成本

本教程使用谷歌云的付费组件：

* Vertex AI
* 谷歌云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 什么是Vertex AI批量预测？

<a href="https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions" target="_blank">批量预测</a> 是一项服务，用于批量处理（即批处理）机器学习推断请求，并且对实时或流式预测的响应时间要求不那么严格。

### 什么是Vertex AI模型监控？
<a href="https://cloud.google.com/vertex-ai/docs/model-monitoring" target="_blank">模型监控</a>是一个服务，可以自动确定生产机器学习流量与训练数据是否不同，或者随着时间的推移在模型预测或特征归因方面变化是否显著。当发生这种情况时，您可以自动和负责任地收到警报，从而可以检测可能会对您的运营产生负面影响的模型衰减，比如客户体验和/或收入。

### 示例模型

在这本笔记本中使用的模型基于[这篇博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。这个模型的背后思想是，你的公司有大量的日志数据描述了游戏用户与网站的互动方式。原始数据包含以下信息类别：

- 身份 - 独特的玩家身份号码
- 人口统计特征 - 有关玩家的信息，例如玩家所在的地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，例如达到新水平
- 流失倾向 - 这是标签或目标特征，它提供了该玩家停止活跃玩家的估计概率。

上面引用的博客文章解释了如何使用BigQuery存储原始数据，对其进行预处理以供机器学习使用，并训练模型。因为这本笔记本侧重于模型监控，而不是训练模型，所以您将重复使用这个模型的预训练版本，该版本已经导出到Google Cloud存储中。在下一节中，您将设置环境并将此模型导入到您自己的项目中。

开始吧

安装Vertex AI SDK for Python，以及其他所需的包。

In [ ]:
# Install Python package dependencies.
! pip3 install --upgrade --quiet google-cloud-aiplatform

! pip3 install --upgrade --quiet tensorflow-data-validation

! pip3 install --quiet cachetools==5.2.0

### 重新启动运行时（仅限Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning"> 
<b>⚠️内核将重新启动。请等待它完成后再继续进行下一步。⚠️</b> 
</div>

### 在Colab上验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type:"string"}

设置您的电子邮件地址
这用于发送模型监控通知。

In [ ]:
import os

EMAIL_ADDRESS = "[your-email-address]"  # @param {type:"string"}
if not EMAIL_ADDRESS or EMAIL_ADDRESS == "[your-email-address]":
    if os.getenv("IS_TESTING"):
        EMAIL_ADDRESS = "noreply@google.com"
    else:
        print("EMAIL_ADDRESS not specified, please correct before proceeding.")

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import time

import google.cloud.aiplatform as aiplatform
import tensorflow_data_validation as tfdv
from google.cloud.aiplatform_v1beta1.services.job_service import \
    JobServiceClient
from google.cloud.aiplatform_v1beta1.types import (
    BatchDedicatedResources, BatchPredictionJob, GcsDestination, GcsSource,
    MachineSpec, ModelMonitoringAlertConfig, ModelMonitoringConfig,
    ModelMonitoringObjectiveConfig, ThresholdConfig)
from tensorflow_data_validation.utils import io_util
from tensorflow_metadata.proto.v0 import statistics_pb2

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### 配置硬件加速器

您可以为预测设置硬件加速器（例如，GPU），或选择不使用任何（CPU）。硬件加速器可以降低预测请求的延迟响应时间。在选择硬件加速器时，请考虑延迟与额外成本的权衡。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机实例的GPU数量。例如，要使用带有4个 Nvidia Tesla T4 GPU的GPU容器映像分配给每个VM，您可以指定：

（aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4，4）

请参阅[可用加速器的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，请指定`(None，None)`以使用在CPU上运行的容器映像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-13"
else:
    DEPLOY_VERSION = "tf2-cpu.2-13"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置预测的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

### 将模型工件上传为 Vertex AI 模型资源

首先，使用`upload()`方法将预训练的自定义表格模型工件作为 Vertex AI 模型资源上传，使用以下参数：

- `display_name`：模型资源的可读名称。
- `artifact_uri`：模型工件的 Cloud Storage 位置。
- `serving_container_image`：模型部署到 Vertex AI 端点资源时要使用的服务容器镜像。
- `sync`：是否等待过程完成，还是立即返回（异步）。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

model = aiplatform.Model.upload(
    display_name="churn",
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

## 使用启用了模型监控功能的批量预测请求。

### 创建云存储桶

这个单元创建了两个云存储桶：

- `gs://PROJECT_ID_bp_mm_input` 包含批量预测请求数据。
- `gs://PROJECT_ID_bp_mm_output` 包含批量预测输出以及模型监控结果。

In [ ]:
# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
OUTPUT_GS_PATH = f"{BUCKET_URI}/bp_mm_output"
INPUT_GS_PATH = f"{BUCKET_URI}/bp_mm_input"
PUBLIC_TRAINING_DATASET = "gs://bp_mm_public_data/churn/churn_bp_insample.csv"
TRAINING_DATASET = f"{INPUT_GS_PATH}/churn_bp_insample.csv"
TRAINING_DATASET_FORMAT = "csv"

! gsutil copy $PUBLIC_TRAINING_DATASET $TRAINING_DATASET

### 创建批量预测作业

这一步参数化并构建代表批量预测请求的数据结构，启用了模型监控。

BatchPredictionJob对象指定输入源、数据格式和用于批量预测的计算资源请求。了解更多关于<a href="https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-sample" target="_blank">BatchPredictionJob</a>。

ModelMonitoringConfig对象指定警报电子邮件地址、训练数据集、要监控的特征以及它们关联的警报阈值。了解更多关于<a href="https://cloud.google.com/vertex-ai/docs/model-monitoring" target="_blank">ModelMonitoringConfig</a>。

In [ ]:
INPUT_URI = "gs://bp_mm_public_data/churn/churn_bp_outsample.jsonl"
OUTPUT_URI = OUTPUT_GS_PATH
INSTANCES_FORMAT = "jsonl"
PREDICTIONS_FORMAT = "jsonl"
JOB_NAME_PREFIX = "bp_mm_demo"
MODEL_NAME = model.resource_name
BATCH_PREDICTION_JOB_NAME = JOB_NAME_PREFIX

batch_prediction_job = BatchPredictionJob(
    display_name=BATCH_PREDICTION_JOB_NAME,
    model=MODEL_NAME,
    input_config=BatchPredictionJob.InputConfig(
        instances_format=INSTANCES_FORMAT, gcs_source=GcsSource(uris=[INPUT_URI])
    ),
    output_config=BatchPredictionJob.OutputConfig(
        predictions_format=PREDICTIONS_FORMAT,
        gcs_destination=GcsDestination(output_uri_prefix=OUTPUT_URI),
    ),
    dedicated_resources=BatchDedicatedResources(
        machine_spec=MachineSpec(machine_type=DEPLOY_COMPUTE),
        starting_replica_count=1,
        max_replica_count=1,
    ),
    # Model monitoring service is triggerred if provide following configs.
    model_monitoring_config=ModelMonitoringConfig(
        alert_config=ModelMonitoringAlertConfig(
            email_alert_config=ModelMonitoringAlertConfig.EmailAlertConfig(
                user_emails=[EMAIL_ADDRESS]
            )
        ),
        objective_configs=[
            ModelMonitoringObjectiveConfig(
                training_dataset=ModelMonitoringObjectiveConfig.TrainingDataset(
                    data_format=TRAINING_DATASET_FORMAT,
                    gcs_source=GcsSource(uris=[TRAINING_DATASET]),
                ),
                training_prediction_skew_detection_config=ModelMonitoringObjectiveConfig.TrainingPredictionSkewDetectionConfig(
                    skew_thresholds={
                        "cnt_user_engagement": ThresholdConfig(value=0.001),
                        "julianday": ThresholdConfig(value=0.001),
                    }
                ),
            )
        ],
    ),
)

提交批量预测作业

该步骤提交在前一步创建的批量预测请求。如果成功，将返回一个JSON文档总结该请求，该文档将显示在下面的单元格输出中。

In [ ]:
API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
client = JobServiceClient(client_options={"api_endpoint": API_ENDPOINT})
out = client.create_batch_prediction_job(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}",
    batch_prediction_job=batch_prediction_job,
)
BATCH_PREDICTION_JOB_ID = out.name.split("/")[-1]
print("BATCH_PREDICTION_JOB_ID:", BATCH_PREDICTION_JOB_ID)

验证预测结果

批量预测请求大约在**17分钟**后完成，模型监控结果则在其之后**10分钟**可用。以下请求获取批量预测作业ID，这是与此类异步请求相关联的唯一编号。

In [ ]:
# If auto-testing, wait for request completion
if os.getenv("IS_TESTING"):
    time.sleep(3000)

浏览存储桶

点击下面的链接，打开上面创建的输出存储桶上的云存储对象查看器，以查看您的批量预测请求的结果。

当一切准备就绪时，您在存储桶中会看到两个文件夹：

- `prediction-batch_prediction_monitoring_test_model_<timestmap>` - 这个文件夹包含您的批量预测结果，即模型为批次中的每个输入生成的预测
- `job-<id>` - 这个文件夹包含模型监控结果，包括模型架构、监控阈值和其他配置设置、统计数据和异常值

*注意：* 在请求和监控作业完成之前，您可能会在此存储桶中看到空白或部分结果。

In [ ]:
print(f"https://console.cloud.google.com/storage/browser/{OUTPUT_URI.lstrip('gs://')}")

### 可视化批量预测结果

运行以下单元格，使用<a href="https://www.tensorflow.org/tfx/data_validation/get_started" target="_blank">TensorFlow数据验证</a>包进行表格和可视化分析来检查批量预测结果。

In [ ]:
! rm -f ./training_stats.pb
! rm -f ./prediction_stats.pb

TRAINING_STATS_SUBPATH = "stats_training/stats/training_stats"
PREDICTION_STATS_SUBPATH = "stats_and_anomalies/stats/current_stats"
STATS_GCS_FOLDER = OUTPUT_URI = (
    OUTPUT_GS_PATH + "/job-" + BATCH_PREDICTION_JOB_ID + "/bp_monitoring/"
)
TRAINING_STATS_GCS_PATH = STATS_GCS_FOLDER + TRAINING_STATS_SUBPATH
print("Looking up statistics from: " + TRAINING_STATS_GCS_PATH)
PREDICTION_STATS_GCS_PATH = STATS_GCS_FOLDER + PREDICTION_STATS_SUBPATH
print("Looking up statistics from: " + PREDICTION_STATS_GCS_PATH)

! gsutil cp $TRAINING_STATS_GCS_PATH ./training_stats.pb
! gsutil cp $PREDICTION_STATS_GCS_PATH ./prediction_stats.pb


# util function to load stats binary file from GCS
def load_stats_binary(input_path):
    stats_proto = statistics_pb2.DatasetFeatureStatisticsList()
    stats_proto.ParseFromString(
        io_util.read_file_to_string(input_path, binary_mode=True)
    )
    return stats_proto


tfdv.visualize_statistics(load_stats_binary("./training_stats.pb"))
tfdv.visualize_statistics(load_stats_binary("./prediction_stats.pb"))

最后，您可以通过检查输出存储桶中的文件来检查偏斜结果。JSON 文件包含了一份报告，显示了批量预测数据在特征之间与训练数据的偏离程度。

In [ ]:
SKEW_GS_PATH = (
    STATS_GCS_FOLDER
    + "stats_and_anomalies/anomalies/training_prediction_skew_anomalies"
)
! gsutil cat $SKEW_GS_PATH

学到更多

**恭喜！**您现在已经学会了如何监控批量预测，以及如何查找和解释结果。查看以下资源，了解更多关于模型监控和ML Ops的内容。

- [TensorFlow数据验证](https://www.tensorflow.org/tfx/guide/tfdv)
- [规模化数据理解、验证和监控](https://blog.tensorflow.org/2018/09/introducing-tensorflow-data-validation.html)
- [Vertex产品文档](https://cloud.google.com/vertex-ai)
- [Vertex AI模型监控参考文档](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.modelDeploymentMonitoringJobs)
- [Vertex AI模型监控博客文章](https://cloud.google.com/blog/topics/developers-practitioners/monitor-models-training-serving-skew-vertex-ai)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Vertex AI 模型
- 批量预测作业
- 云存储存储桶（将 `delete_bucket` 设置为 *True* 进行删除）

In [ ]:
# Delete model resource
model.delete()

# Get the batch prediction job resource
batch_job = aiplatform.BatchPredictionJob(
    batch_prediction_job_name=BATCH_PREDICTION_JOB_ID
)
# Delete the job
batch_job.delete()

# Delete Cloud Storage bucket
delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

! rm -f ./training_stats.pb
! rm -f ./prediction_stats.pb